## LangChain with time tool
> **Expected Output**: Action: today's date

LangChain agent with time tool.  The tool is defined with a LangChain decorator and registered with the initialized agent.


In [6]:
import os
import requests
import json
import httpx
from openai import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain_core.tools import Tool
from langchain.tools import tool
from datetime import date
import warnings

In [7]:
warnings.filterwarnings('ignore')
httpx_client = httpx.Client(http2=True, verify=False, timeout=10.0)

vcapservices = os.getenv('VCAP_SERVICES')
services = json.loads(vcapservices)

def is_chatservice(service):
    return service["name"] == "gen-ai-qwen3-ultra"

chat_services = filter(is_chatservice, services["genai"])
chat_credentials = list(chat_services)[0]["credentials"]

llm = ChatOpenAI(temperature=0.9, model=chat_credentials["model_name"], base_url=chat_credentials["api_base"], api_key=chat_credentials["api_key"], http_client=httpx_client)


In [8]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())


# list of tools for agent
tools = [time]

In [9]:
# initialize agent with tools and llm
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")




> Entering new AgentExecutor chain...
<think>
Okay, the user is asking for today's date. Let me check what tools I have. Oh right, there's the 'time' tool that returns today's date. I need to use that. The input for the time function should be an empty string. So I'll structure the JSON blob with the action as 'time' and an empty action_input. Then I'll get the observation, which will be today's date. Finally, I'll present that as the answer.
</think>

Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2025-08-14
Thought:<think>
Okay, the user asked for today's date. I used the time tool, which returned 2025-08-14. That's the current date. I need to present this as the final answer. Make sure to use the exact format "Final Answer" followed by the date in quotes.

Wait, the observation is 2025-08-14. So the answer should be "2025-08-14". Let me check if there's any formatting needed. The user just wants the date, so that's it.

Final Answer: "2025-08-14"

> Fin